In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import czifile as zis
from apeer_ometiff_library import io, processing, omexmlClass
import os
from matplotlib import pyplot as plt, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.viewer import ImageViewer
import skimage.io
import matplotlib.colors as colors
import numpy as np
#from skimage.external import tifffile
import ipywidgets as widgets

import imgfileutils as imf

In [ ]:
#basefolder = r'/datadisk1/tuxedo/IPython_Notebooks/testdata'
#basefolder = r'/home/sebi06/testdata'
basefolder = r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV'

#filename = os.path.join(basefolder, 'Filter_with_Particles_big.ome.tiff')
#filename = os.path.join(basefolder, 'S=2_T=5_CH=3_CH=2_A2.ome.tiff')
#filename = os.path.join(basefolder, 'Osteosarcoma_01.ome.tiff')
#filename = os.path.join(basefolder, 'Filter_with_Particles_small.czi')
#filename = os.path.join(basefolder, '8Brains_DAPI_5X_stitched.czi')
#filename = os.path.join(basefolder, r'2x2_SNAP_CH=2_Z=5_T=2.czi')
#filename = os.path.join(basefolder, 'S=2_T=5_Z=3_CH=2_A2.czi')
filename = os.path.join(basefolder, r'CellDivision_T=15_Z=20_CH=2_DCV.czi')

In [ ]:
image_name = os.path.basename(filename)

if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):
    
    metadata = imf.get_metadata(filename)
    
    print(metadata['Shape'])
    print(metadata['Axes'])

    
    array, metadata['DimOrder CZI'] = imf.get_array_czi(filename,
                                                        cziaxes=metadata['Axes'],
                                                        blockindex=0,
                                                        sceneindex=0,
                                                        replacezero=False)
    print(array.shape)

In [ ]:
print(array.min(), array.max())
print(metadata['Shape'])
print(metadata['Axes'])
print(array.shape)
print(metadata['Extension'])

In [ ]:
#out = widgets.interactive_output(get_TZC, { 't': t, 'z': z, 'c': c, 'r':r})
if metadata['Extension'] == 'ome.tiff':
    ui, out = imf.create_ipyviewer_ometiff(array, metadata)
if metadata['Extension'] == 'czi':
    ui, out = imf.create_ipyviewer_czi(array, metadata)

display(ui, out)

In [ ]:
def get_dimorder(dimstring):
    
    dimindex_list = []
    dims = ['B', 'S', 'T', 'C', 'Z', 'Y', 'X', '0']
    dims_dict = {}
    
    for d in dims:
    
        dims_dict[d] = dimstring.find(d)
        dimindex_list.append(dimstring.find(d))
        
    numvalid_dims = sum(i > 0 for i in dimindex_list)
    
    return dims_dict, dimindex_list, numvalid_dims

dim_dict, dim_list, numvalid_dims = get_dimorder(metadata['Axes'])

print(dim_dict)
print(dim_list)
print(numvalid_dims)

In [ ]:
if dim_dict['B'] < 0 and dim_dict['S'] >=0:
    # B does not exist but S does
    cut = (0, count)
if dim_dict['B'] >= 0 and dim_dict['S'] >=0:
    # B and S do exist
    cut = (0, 1, count)
if dim_dict['B'] < 0 and dim_dict['S'] < 0:
    # B and S do not exist
    cut = (count)
if dim_dict['B'] >= 0 and dim_dict['S'] <0:
    # B does exist but S does not
    cut = (0, count)
    
print(cut)

In [ ]:
out = np.squeeze(array, axis=(count))
print(array.shape)
print(out.shape)